### Por qué la regla de Hebb **pura** no aprende la compuerta AND

**Idea clave:** Hebb refuerza coactivaciones pero **no corrige errores** ni **ajusta el sesgo**. Para AND, eso hace que (1,0) y (0,1) queden mal clasificadas como 1.

**Definición (Hebb clásica, supervisada):**
Datos $x\in\{0,1\}^2$, etiqueta $y\in\{0,1\}$. Actualización:

$$
\Delta w=\eta\, x\, y,\qquad \Delta b=0,
$$

con neurona $ \hat y=\mathbb{1}[\,w^\top x + b>0\,]$.

**Conjunto AND:**

$$
(0,0)\mapsto 0,\quad (0,1)\mapsto 0,\quad (1,0)\mapsto 0,\quad (1,1)\mapsto 1.
$$

**Tras una pasada por los datos:**
Solo $(1,1)$ tiene $y=1$, por tanto

$$
w=\eta(1,1)^\top,\qquad b=0.
$$

Entonces

$$
w^\top(1,0)=\eta>0,\quad w^\top(0,1)=\eta>0,\quad w^\top(1,1)=2\eta>0,
$$

lo que produce $\hat y=1$ para $(1,0)$ y $(0,1)$, que es **incorrecto**.

**Qué faltó:** Para que AND funcione se requiere un **sesgo negativo** que suba el umbral efectivo:

$$
\text{aceptar solo }(1,1)\ \Longleftrightarrow\ b\in(-2\eta,\,-\eta).
$$

La regla Hebb pura **no** ajusta $b$ ni reduce pesos ante falsos positivos, así que no puede “aprender” AND por sí sola.

**Conclusión:** Hebb pura falla en AND. Funciona si:

1. se **impone** un sesgo $b$ adecuado o una entrada constante $x_0\equiv 1$ con peso inhibitorio aprendido, o
2. se usa una regla con **corrección de error** (perceptrón, delta rule) que ajuste $w,b$.

In [1]:
import numpy as np

# Datos AND
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,0,0,1])

eta = 1.0

# Hebb pura (sin sesgo)
w = np.zeros(2)
for xi, yi in zip(X, y):
    w += eta * xi * yi  # solo (1,1) suma
b = 0.0

def predict(X, w, b):
    return (X @ w + b > 0).astype(int)

print("Pesos Hebb:", w, "bias:", b)
print("Predicción sin sesgo:", predict(X, w, b))  # Falla en (1,0) y (0,1)

# Fijar un sesgo negativo adecuado
b_ok = -1.5 * eta  # en (-2*eta, -eta)
print("Predicción con sesgo negativo:", predict(X, w, b_ok))  # Ahora es AND correcto


Pesos Hebb: [1. 1.] bias: 0.0
Predicción sin sesgo: [0 1 1 1]
Predicción con sesgo negativo: [0 0 0 1]


Sí puedes interpretarla y, de hecho, tu tabla muestra **que con sesgo** la Hebb supervisada **sí** puede implementar AND. El “no puede” clásico se refiere a Hebb **sin sesgo**.

### Cómo leer tu tabla

* Tres primeras columnas (izq., resaltadas): $x_1,\ x_2,\ t\in\{+1,-1\}$ con codificación bipolar. Para AND: $t=+1$ solo cuando $(x_1,x_2)=(+1,+1)$; en los demás casos $t=-1$.
* Tres columnas derechas: acumulados de la regla Hebb supervisada

  $$
  \Delta w = t\,x,\quad \Delta b = t,
  $$

  por lo que después de cada fila:

  $$
  w_1 \leftarrow w_1 + t\,x_1,\qquad
  w_2 \leftarrow w_2 + t\,x_2,\qquad
  b \leftarrow b + t.
  $$
* Clasificador resultante: $\hat y=\mathrm{sign}(w_1 x_1 + w_2 x_2 + b)$.

### Qué está pasando en tus números

* Observas $w_1\approx w_2$ creciendo y $b$ volviéndose **más negativo** (suma de muchas etiquetas $-1$).
* En algún punto, por ejemplo con $(w_1,w_2,b)=(4,4,-4)$, las puntuaciones son:

  $$
  s(+1,+1)=2\cdot4-4=4>0,\quad
  s(+1,-1)=0-4=-4<0,\quad
  s(-1,+1)=0-4=-4<0,\quad
  s(-1,-1)=-8-4<0,
  $$

  que coincide exactamente con AND en bipolar: solo $(+1,+1)$ es positivo.

### Conclusión operativa

* **Con sesgo aprendido** vía una entrada constante $x_0\equiv 1$ y $\Delta w_0=\Delta b=t$, Hebb **sí** separa AND.
* **Sin sesgo** ($\Delta b=0$) Hebb fija $b=0$ y queda $w\propto(1,1)$; entonces $w^\top(1,0)>0$ y $w^\top(0,1)>0$, lo que **siempre** falla para AND.
  Ese es el caso al que se alude cuando se dice que “Hebb no puede AND”.

  Sí. Se “separan” en el **espacio proyectado** por $w$, no en el plano original.

* Modelo: $\hat t=\operatorname{sign}(s(x))$, con $s(x)=w^\top x+b$.
* Hebb (bipolar): $\Delta w=t\,x,\ \Delta b=t$. Cada ejemplo mueve $w$ **hacia la correlación** entrada–etiqueta.

Lectura de signos y magnitudes:

* $w_i$ **positivo grande** ⇒ la entrada $x_i=+1$ **empuja** $s(x)$ a $+$ (favorece $t=+1$); $x_i=-1$ la **penaliza**.
* $w_i$ **negativo grande** ⇒ $x_i=+1$ penaliza y $x_i=-1$ favorece.
* $b$ **negativo** desplaza el umbral para volver más **exigente** la activación.

Qué implica “más positivo/negativo”:

* Aumenta $|s(x)|$ y por tanto el **margen** $\gamma=\min_k \frac{t_k\,s(x_k)}{\|w\|}$. Si tus updates alinean $w$ con el patrón positivo, $\gamma$ crece y la clasificación se vuelve **robusta**.
* En tu tabla, muchos $t=-1$ ⇒ $b$ se hace muy negativo. Eso fuerza a que solo $(+1,+1)$ logre $s(x)>0$:

  $$
  s(+1,+1)=w_1+w_2+b,\quad
  s(+1,-1)=w_1-w_2+b,\quad
  s(-1,+1)=-w_1+w_2+b,\quad
  s(-1,-1)=-w_1-w_2+b.
  $$

  Con $w_1\approx w_2>0$ y $b\ll0$, solo el primero queda $>\!0$ ⇒ AND.

Notas:

* Sí, crecer en magnitud **separa** las clases tras la proyección $x\mapsto s(x)$.
* Hebb puede hacer que $\|w\|$ “explote”; la **dirección** de $w$ y el **signo** de $s(x)$ son lo relevante. Normalizar $w$ o parar por margen evita explosión.

